In [1]:
#import base libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [2]:
#load data
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

## Outlier Detection

In [1]:
#function for returning rows with outliers

def detect_outliers(df,n,features):
    """
    Takes a data frame df of features and returns a list of indices
    corresponding to the observations containing more than n outlier according to the Tunkey Method
    """
    outlier_indices=[]
    
    #iterate over features
    
    for col in features:
        #1st quartile (25%)
        Q1=np.percentile(df[col],25)
        #3rd quartile (75%)
        Q3=np.percentile(df[col,75])
        #interquartile range(IQR)
        IQR=Q3-Q1
        
        #outlier step
        outlier_step=1.5*IQR
        criteria=(df[col]<Q1-outlier_step)|(df[col]>Q1+outlier_step)        
        #applying boolean mask to filter rows and get the indices
        outlier_list_col = df[criteria].index
        # append to list
        outlier_indices.extend(outlier_list_col)
        # intantiate a counter object from list
        outlier_indices=Counter(outlier_list_col)
        #flatten the counter itesm which are in from of tuple (a,b) 
        #and to multiple outliers if number of outliers in a row exceed 1, ie 2 or more
        multiple_outliers = list( k for k, v in outlier_indices.items() if v > 1 )
        
        return multiple_outliers
        
